<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/ehime_mymap_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas
!pip install rtree
!pip install pygeos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 48.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.3 MB/s eta 0:00:00


In [2]:
import zipfile

In [3]:
import requests
import pandas as pd
import geopandas as gpd
from lxml import etree

/usr/local/lib/python3.9/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-3-d5a31a737340>:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  impo

In [4]:
def enblcid_split(df0):

    df1 = df0["eNB-LCID"].str.split("-", expand=True).rename(columns={0: "eNB", 1: "LCID"})
    df2 = df0.join(df1)
    
    df2["LCID"] = df2["LCID"].str.split(",")
    df3 = df2.explode("LCID").astype({"eNB": int, "LCID": int})
    
    df4 = df3.sort_values(["eNB", "LCID"]).reset_index(drop=True)

    df4["id"] = df4["eNB"].astype(str) + "-" + df4["LCID"].astype(str)

    return df4

# 設定

In [5]:
# マイマップのmid
mid = "152eagLReFE8oDs_thOuVxxXSx8IhegsS"
lid = "TPK-JtNO-wk"

# マイマップからKMZをダウンロード

In [6]:
url = f"https://www.google.com/maps/d/kml?mid={mid}&lid={lid}"

In [7]:
!wget "$url" -O rakuten.kmz

--2023-04-26 08:19:37--  https://www.google.com/maps/d/kml?mid=152eagLReFE8oDs_thOuVxxXSx8IhegsS&lid=TPK-JtNO-wk
Resolving www.google.com (www.google.com)... 142.250.136.104, 142.250.136.105, 142.250.136.106, ...
Connecting to www.google.com (www.google.com)|142.250.136.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.google-earth.kmz]
Saving to: ‘rakuten.kmz’

rakuten.kmz             [ <=>                ] 525.41K  --.-KB/s    in 0.007s  

2023-04-26 08:19:38 (78.4 MB/s) - ‘rakuten.kmz’ saved [538023]



# KMZからCSV変換

In [8]:
with zipfile.ZipFile("rakuten.kmz") as existing_zip:
    existing_zip.extractall("kmz")

In [9]:
ns = {"kml": "http://www.opengis.net/kml/2.2"}
tree = etree.parse("./kmz/doc.kml")

In [10]:
data = []

for i in tree.xpath("//kml:Placemark", namespaces=ns):

    d = {}

    d["場所"] = "".join(i.xpath("./kml:name/text()", namespaces=ns)).strip()
    d["geometry"] = "".join(
        i.xpath("./kml:Point/kml:coordinates/text()", namespaces=ns)
    ).strip()

    for j in i.xpath("./kml:ExtendedData/kml:Data", namespaces=ns):

        k = j.attrib["name"].strip()
        v = "".join(j.xpath("./kml:value/text()", namespaces=ns)).strip()

        d[k] = v

    data.append(d)

In [11]:
df0 = pd.DataFrame(data)

In [12]:
df0.shape

(699, 20)

In [13]:
df0

,場所,geometry,【業務連絡】,基地局番号 愛媛-,市町村,セクタ数,eNB-LCID,PCI,開局/未開局,回線種別,開局/未開局【5G設備】,【5G】開局/未開局,Twitterのリンク,情報提供等,(年月日),二次情報提供,情報提供リンク,説明,未整理Tweet,gx_media_links
0,松山市東方町甲,"132.8152126,33.7713417,0",,53.0,松山市,3,"737292-4,5,6","221,219,220",開局,,,,https://twitter.com/kmtt2012/status/1354658933...,発見･写真:@kmtt2012,2021/1/28,,,,,https://doc-00-88-mymaps.googleusercontent.com...
1,松山市平和通3丁目,"132.7657555,33.8478096,0",,69.0,松山市,2,"737302-1,2,3","301,300,302",開局,,,,https://twitter.com/kmtt2012/status/1355770457...,発見･写真:@kmtt2012,2021/1/31,,,四国通建ビル,,https://doc-00-88-mymaps.googleusercontent.com...
2,松山市清住2丁目,"132.7166716,33.8536943,0",,34.0,松山市,3,"737282-7,8,9","140,138,139",開局,,,,https://twitter.com/kmtt2012/status/1354719726...,発見･写真:@kmtt2012,2021/1/28,,,マルナカ清住店近く,,https://doc-08-88-mymaps.googleusercontent.com...
3,松山市鷹子町,"132.8110742,33.815409,0",,44.0,松山市,3,"737284-4,5,6","6,308,7",開局,,,,https://twitter.com/kmtt2012/status/1354686943...,発見･写真:@kmtt2012,2021/1/28,,,極楽寺,,https://doc-14-88-mymaps.googleusercontent.com...
4,松山市中野町,"132.8327865,33.7776925,0",,49.0,松山市,3,"737292-1,2,3","35,34,33",開局,,,,https://twitter.com/kmtt2012/status/1354663880...,発見･写真:@kmtt2012,2021/1/28,,,ローソン松山中野町上ノ原店近く,,https://doc-0k-88-mymaps.googleusercontent.com...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,（仮）伊予市双海町上灘⑤,"132.688375,33.6805,0",未整理,,大洲市,2,,,未開局,,,,https://twitter.com/saiseki56/status/164929352...,発見・写真:@saiseki56,2023/04/21,,,仮登録,https://twitter.com/saiseki56/status/165040750...,NaN
695,上浮穴郡久万高原町黒藤川,"133.03623,33.605406,0",未整理,780.0,久万高原町,1,742240-1,1(4/25),開局(衛星),,,,,発見・写真:@sfuruie0407,2023/4/25,,https://twitter.com/sfuruie0407/status/1650825...,,,NaN
696,東温市上村,"132.848807,33.776383,0",未整理(誤消去の為後で復旧),781.0,東温市,3,"737386-1,2,3","319,171,8(4/23)",開局,,,,,発見・写真:@sfuruie0407,2023/4/23,,https://twitter.com/sfuruie0407/status/1650107...,,https://twitter.com/kmtt2012/status/1651029713...,NaN
697,東温市上林②,"132.882877,33.758991,0",未整理,782.0,東温市,3,"737372-7,8,9","5,87,187(4/23)",開局,,,,,,2023/4/23,発見・写真:@sfuruie0407,https://twitter.com/sfuruie0407/status/1650105...,,,NaN


In [14]:
df0.to_csv("all.csv")

In [15]:
df0[["経度", "緯度", "標高"]] = df0["geometry"].str.split(",", expand=True)

In [16]:
# 文字から数値に変換
df0["経度"] = pd.to_numeric(df0["経度"])
df0["経度"] = pd.to_numeric(df0["経度"])

In [17]:
# 緯度・経度が欠損の場合は削除
df0.dropna(subset=["緯度", "経度"], how="any", inplace=True)
df0

,場所,geometry,【業務連絡】,基地局番号 愛媛-,市町村,セクタ数,eNB-LCID,PCI,開局/未開局,回線種別,...,情報提供等,(年月日),二次情報提供,情報提供リンク,説明,未整理Tweet,gx_media_links,経度,緯度,標高
0,松山市東方町甲,"132.8152126,33.7713417,0",,53.0,松山市,3,"737292-4,5,6","221,219,220",開局,,...,発見･写真:@kmtt2012,2021/1/28,,,,,https://doc-00-88-mymaps.googleusercontent.com...,132.815213,33.7713417,0
1,松山市平和通3丁目,"132.7657555,33.8478096,0",,69.0,松山市,2,"737302-1,2,3","301,300,302",開局,,...,発見･写真:@kmtt2012,2021/1/31,,,四国通建ビル,,https://doc-00-88-mymaps.googleusercontent.com...,132.765756,33.8478096,0
2,松山市清住2丁目,"132.7166716,33.8536943,0",,34.0,松山市,3,"737282-7,8,9","140,138,139",開局,,...,発見･写真:@kmtt2012,2021/1/28,,,マルナカ清住店近く,,https://doc-08-88-mymaps.googleusercontent.com...,132.716672,33.8536943,0
3,松山市鷹子町,"132.8110742,33.815409,0",,44.0,松山市,3,"737284-4,5,6","6,308,7",開局,,...,発見･写真:@kmtt2012,2021/1/28,,,極楽寺,,https://doc-14-88-mymaps.googleusercontent.com...,132.811074,33.815409,0
4,松山市中野町,"132.8327865,33.7776925,0",,49.0,松山市,3,"737292-1,2,3","35,34,33",開局,,...,発見･写真:@kmtt2012,2021/1/28,,,ローソン松山中野町上ノ原店近く,,https://doc-0k-88-mymaps.googleusercontent.com...,132.832786,33.7776925,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,（仮）伊予市双海町上灘⑤,"132.688375,33.6805,0",未整理,,大洲市,2,,,未開局,,...,発見・写真:@saiseki56,2023/04/21,,,仮登録,https://twitter.com/saiseki56/status/165040750...,NaN,132.688375,33.6805,0
695,上浮穴郡久万高原町黒藤川,"133.03623,33.605406,0",未整理,780.0,久万高原町,1,742240-1,1(4/25),開局(衛星),,...,発見・写真:@sfuruie0407,2023/4/25,,https://twitter.com/sfuruie0407/status/1650825...,,,NaN,133.036230,33.605406,0
696,東温市上村,"132.848807,33.776383,0",未整理(誤消去の為後で復旧),781.0,東温市,3,"737386-1,2,3","319,171,8(4/23)",開局,,...,発見・写真:@sfuruie0407,2023/4/23,,https://twitter.com/sfuruie0407/status/1650107...,,https://twitter.com/kmtt2012/status/1651029713...,NaN,132.848807,33.776383,0
697,東温市上林②,"132.882877,33.758991,0",未整理,782.0,東温市,3,"737372-7,8,9","5,87,187(4/23)",開局,,...,,2023/4/23,発見・写真:@sfuruie0407,https://twitter.com/sfuruie0407/status/1650105...,,,NaN,132.882877,33.758991,0


In [18]:
df0["場所"] = df0["場所"].str.replace("【.+】", "", regex=True)

In [19]:
df0["eNB-LCID"] = df0["eNB-LCID"].str.strip()

In [20]:
df1 = df0[df0["eNB-LCID"].str.startswith("73")].copy()

In [21]:
df1 = df1[["場所", "市町村", "セクタ数", "eNB-LCID", "PCI", "緯度", "経度"]]

In [22]:
df2 = enblcid_split(df1)

In [23]:
df2.to_csv("list.csv", encoding="utf_8_sig")

In [24]:
df2

,場所,市町村,セクタ数,eNB-LCID,PCI,緯度,経度,eNB,LCID,id
0,松山市三津3丁目,松山市,3,"737282-1,2,3","23,22,21",33.862073,132.714348,737282,1,737282-1
1,松山市三津3丁目,松山市,3,"737282-1,2,3","23,22,21",33.862073,132.714348,737282,2,737282-2
2,松山市三津3丁目,松山市,3,"737282-1,2,3","23,22,21",33.862073,132.714348,737282,3,737282-3
3,松山市山西町,松山市,3,"737282-4,5,6","24,25,26",33.849767,132.733195,737282,4,737282-4
4,松山市山西町,松山市,3,"737282-4,5,6","24,25,26",33.849767,132.733195,737282,5,737282-5
...,...,...,...,...,...,...,...,...,...,...
1679,大洲市阿蔵①,大洲市,3,"737579-1,2,3","86,85,84(4/16)",33.5085865,132.536129,737579,2,737579-2
1680,大洲市阿蔵①,大洲市,3,"737579-1,2,3","86,85,84(4/16)",33.5085865,132.536129,737579,3,737579-3
1681,北宇和郡鬼北町上川,鬼北町,1,737579-4,20(4/23),33.264205,132.724750,737579,4,737579-4
1682,大洲市春賀②,大洲市,2,"737579-5,6","81,82(4/17)",33.563244,132.554939,737579,5,737579-5


# 電測マップ

In [25]:
# スプレッドシートのURL
csv_url = "https://raku10ehime.github.io/map/ehime.csv"

In [26]:
df3 = pd.read_csv(csv_url, index_col=0).dropna(how="all").rename(columns={"市区町村": "市町村", "sector": "セクタ数"})
df3

,場所,市町村,場所補足,設置タイプ,状況,セクタ数,sub6,ミリ波,eNB-LCID,PCI,基地局ID,緯度,経度,更新日時,color,icon
0,【5G】今治市立花町二丁目,今治市,NaN,コン柱,open,3.0,3.0,3.0,"737335-1,2,3","39,347,139",NaN,34.052319,132.999271,2022/06/01 0:00:00,darkblue,plane
1,【5G】今治市中寺①,今治市,今治中寺郵便局付近,コン柱,open,3.0,3.0,3.0,"737331-1,2,3","292,98,300",NaN,34.037485,132.991341,2022/06/01 0:00:00,darkblue,plane
2,今治市桜井四丁目,今治市,志島ヶ原・綱敷天満神社付近,コン柱,open,3.0,NaN,NaN,"737512-10,11,12","47,378,94",NaN,34.017394,133.041464,2023/04/23 16:44:15,green,signal
3,今治市登畑,今治市,レデイ薬局今治国分店付近,コン柱,open,3.0,NaN,NaN,"737510-4,5,6","136,278,135",NaN,34.019522,133.016295,2023/04/23 18:22:03,green,signal
4,【5G】今治市阿方①,今治市,ファミリーマート今治阿方店付近,コン柱,open,3.0,1.0,2.0,"737354-4,5,6","225,227,226",NaN,34.061198,132.970841,2022/11/13 2:31:56,darkblue,plane
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,西予市明浜町高山,西予市,NaN,コン柱,open,3.0,NaN,NaN,"737523-7,8,9","38,181,36",CBSB838000416,33.321618,132.433275,2023/04/22 16:59:01,green,signal
698,西予市三瓶町垣生,西予市,NaN,コン柱,open,1.0,NaN,NaN,737575-5,10,NaN,33.386473,132.409811,2023/04/22 17:28:16,green,signal
699,今治市小浦町,今治市,サンライズ糸山,コン柱,open,3.0,NaN,NaN,"737378-10,11,12","14,145,96",NaN,34.109348,132.978349,2023/04/23 16:08:12,green,signal
700,東温市上林②,東温市,NaN,コン柱,open,3.0,NaN,NaN,"737372-7,8,9","5,87,187",NaN,33.758991,132.882877,2023/04/23 21:53:23,green,signal


In [27]:
df3["場所"] = df3["場所"].str.replace("【.+】", "", regex=True)

In [28]:
df3["eNB-LCID"] = df3["eNB-LCID"].fillna("")

In [29]:
df3[df3["eNB-LCID"] != df3["eNB-LCID"].str.strip()]

,場所,市町村,場所補足,設置タイプ,状況,セクタ数,sub6,ミリ波,eNB-LCID,PCI,基地局ID,緯度,経度,更新日時,color,icon


In [30]:
df4 = df3[df3["eNB-LCID"].str.strip().str.startswith("73")].copy()

In [31]:
df5 = df4[["場所", "市町村", "セクタ数", "eNB-LCID", "PCI", "緯度", "経度"]].copy()

In [32]:
df6 = enblcid_split(df5)

# 結合

In [33]:
df7 = pd.merge(df2, df6, on="id", how="outer")

In [34]:
df7

,場所_x,市町村_x,セクタ数_x,eNB-LCID_x,PCI_x,緯度_x,経度_x,eNB_x,LCID_x,id,場所_y,市町村_y,セクタ数_y,eNB-LCID_y,PCI_y,緯度_y,経度_y,eNB_y,LCID_y
0,松山市三津3丁目,松山市,3,"737282-1,2,3","23,22,21",33.862073,132.714348,737282.0,1.0,737282-1,松山市三津三丁目,松山市,3.0,"737282-1,2,3","23,22,21",33.862073,132.714348,737282.0,1.0
1,松山市三津3丁目,松山市,3,"737282-1,2,3","23,22,21",33.862073,132.714348,737282.0,2.0,737282-2,松山市三津三丁目,松山市,3.0,"737282-1,2,3","23,22,21",33.862073,132.714348,737282.0,2.0
2,松山市三津3丁目,松山市,3,"737282-1,2,3","23,22,21",33.862073,132.714348,737282.0,3.0,737282-3,松山市三津三丁目,松山市,3.0,"737282-1,2,3","23,22,21",33.862073,132.714348,737282.0,3.0
3,松山市山西町,松山市,3,"737282-4,5,6","24,25,26",33.849767,132.733195,737282.0,4.0,737282-4,松山市山西町,松山市,3.0,"737282-4,5,6","24,25,26",33.849767,132.733195,737282.0,4.0
4,松山市山西町,松山市,3,"737282-4,5,6","24,25,26",33.849767,132.733195,737282.0,5.0,737282-5,松山市山西町,松山市,3.0,"737282-4,5,6","24,25,26",33.849767,132.733195,737282.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737529-4,上浮穴郡久万高原町直瀬,久万高原町,3.0,"737529-2,3,4","324,233,231",33.706192,132.959651,737529.0,4.0
1784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737545-10,西条市丹原町長野,西条市,3.0,"737545-10,11,12","151,152,150",33.876367,133.042725,737545.0,10.0
1785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737545-11,西条市丹原町長野,西条市,3.0,"737545-10,11,12","151,152,150",33.876367,133.042725,737545.0,11.0
1786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737545-12,西条市丹原町長野,西条市,3.0,"737545-10,11,12","151,152,150",33.876367,133.042725,737545.0,12.0


# 電測マップにない

In [35]:
# マイマップ
df7[df7["eNB-LCID_y"].isna()]

,場所_x,市町村_x,セクタ数_x,eNB-LCID_x,PCI_x,緯度_x,経度_x,eNB_x,LCID_x,id,場所_y,市町村_y,セクタ数_y,eNB-LCID_y,PCI_y,緯度_y,経度_y,eNB_y,LCID_y
425,今治市桜井4丁目,今治市,3,"737342-7,8,9","239,222,100",34.017402,133.041459,737342.0,7.0,737342-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,今治市桜井4丁目,今治市,3,"737342-7,8,9","239,222,100",34.017402,133.041459,737342.0,8.0,737342-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,今治市桜井4丁目,今治市,3,"737342-7,8,9","239,222,100",34.017402,133.041459,737342.0,9.0,737342-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,今治市古国分1丁目,今治市,2,"737344-10,11","221,7",34.0319829,133.036283,737344.0,10.0,737344-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,今治市古国分1丁目,今治市,2,"737344-10,11","221,7",34.0319829,133.036283,737344.0,11.0,737344-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,西予市城川町高野子,西予市,3,"737549-9,10,11","327,328,329(4/4)",33.363006,132.797986,737549.0,9.0,737549-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1579,西予市城川町高野子,西予市,3,"737549-9,10,11","327,328,329(4/4)",33.363006,132.797986,737549.0,10.0,737549-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1580,西予市城川町高野子,西予市,3,"737549-9,10,11","327,328,329(4/4)",33.363006,132.797986,737549.0,11.0,737549-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1584,北宇和郡鬼北町川上,,,737551-10,20(4/9),33.305662,132.759682,737551.0,10.0,737551-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# マイマップにない

In [36]:
# 電測マップ
df7[df7["eNB-LCID_x"].isna()]

,場所_x,市町村_x,セクタ数_x,eNB-LCID_x,PCI_x,緯度_x,経度_x,eNB_x,LCID_x,id,場所_y,市町村_y,セクタ数_y,eNB-LCID_y,PCI_y,緯度_y,経度_y,eNB_y,LCID_y
1692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737346-4,西条市坂元甲,西条市,3.0,"737346-4,5,6","246,247,248",33.893630,133.151582,737346.0,4.0
1693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737346-5,西条市坂元甲,西条市,3.0,"737346-4,5,6","246,247,248",33.893630,133.151582,737346.0,5.0
1694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737346-6,西条市坂元甲,西条市,3.0,"737346-4,5,6","246,247,248",33.893630,133.151582,737346.0,6.0
1695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737350-1,四国中央市川之江町②,四国中央市,3.0,"737350-1,2,3","87,88,89",34.018855,133.576226,737350.0,1.0
1696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737350-2,四国中央市川之江町②,四国中央市,3.0,"737350-1,2,3","87,88,89",34.018855,133.576226,737350.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737529-4,上浮穴郡久万高原町直瀬,久万高原町,3.0,"737529-2,3,4","324,233,231",33.706192,132.959651,737529.0,4.0
1784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737545-10,西条市丹原町長野,西条市,3.0,"737545-10,11,12","151,152,150",33.876367,133.042725,737545.0,10.0
1785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737545-11,西条市丹原町長野,西条市,3.0,"737545-10,11,12","151,152,150",33.876367,133.042725,737545.0,11.0
1786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737545-12,西条市丹原町長野,西条市,3.0,"737545-10,11,12","151,152,150",33.876367,133.042725,737545.0,12.0


# 距離

In [37]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import pyproj

In [39]:
df8 = df7.dropna(subset=["eNB-LCID_x", "eNB-LCID_y"], how="any").copy()

# eNB-LCIDが違う

In [40]:
df8[df8["eNB-LCID_x"] != df8["eNB-LCID_y"]]

,場所_x,市町村_x,セクタ数_x,eNB-LCID_x,PCI_x,緯度_x,経度_x,eNB_x,LCID_x,id,場所_y,市町村_y,セクタ数_y,eNB-LCID_y,PCI_y,緯度_y,経度_y,eNB_y,LCID_y
1099,四国中央市妻鳥町①,四国中央市,3,"737459-7,8,9","254,10,258(2/23)",33.98302268,133.575783,737459.0,7.0,737459-7,北宇和郡鬼北町大字下鍵山②,鬼北町,2.0,"737459-7,8","243,245",33.329938,132.799157,737459.0,7.0
1101,四国中央市妻鳥町①,四国中央市,3,"737459-7,8,9","254,10,258(2/23)",33.98302268,133.575783,737459.0,8.0,737459-8,北宇和郡鬼北町大字下鍵山②,鬼北町,2.0,"737459-7,8","243,245",33.329938,132.799157,737459.0,8.0
1649,八幡浜市矢野町,八幡浜市,3,"737567-8,9","376,377(3/19)",33.4572,132.424473,737567.0,8.0,737567-8,八幡浜市矢野町,八幡浜市,3.0,"737567-7,8,9","375,376,377",33.457200,132.424473,737567.0,8.0
1650,八幡浜市矢野町,八幡浜市,3,"737567-8,9","376,377(3/19)",33.4572,132.424473,737567.0,9.0,737567-9,八幡浜市矢野町,八幡浜市,3.0,"737567-7,8,9","375,376,377",33.457200,132.424473,737567.0,9.0


In [41]:
df9 = df8.drop_duplicates(subset="eNB-LCID_x").copy()

In [42]:
grs80 = pyproj.Geod(ellps="GRS80")

In [43]:
df9["距離"] = df9.apply(lambda x: grs80.inv(x["経度_x"], x["緯度_x"], x["経度_y"], x["緯度_y"])[2], axis=1)

# 10m以上違う

In [44]:
df10 = df9[df9["距離"] > 10].sort_values("距離", ascending=False).copy()

In [45]:
df10

,場所_x,市町村_x,セクタ数_x,eNB-LCID_x,PCI_x,緯度_x,経度_x,eNB_x,LCID_x,id,場所_y,市町村_y,セクタ数_y,eNB-LCID_y,PCI_y,緯度_y,経度_y,eNB_y,LCID_y,距離
441,新居浜市上原2丁目,新居浜市,3,"737344-4,5,6","156,158,157",33.922277,133.284421,737344.0,4.0,737344-4,新居浜市上原二丁目,新居浜市,3.0,"737344-4,5,6","156,158,157",33.921289,133.284536,737344.0,4.0,110.139342
1587,新居浜市郷2丁目,新居浜市,3,"737554-1,2,3","204,62,259(4/16)",33.966087,133.309993,737554.0,1.0,737554-1,新居浜市郷二丁目,新居浜市,3.0,"737554-1,2,3","204,62,259",33.965556,133.309820,737554.0,1.0,61.031085
1438,（仮）西予市宇和町田野中,西予市,3,"737525-1,2,3","%75,376,377(4/17)",33.4020981,132.537272,737525.0,1.0,737525-1,西予市宇和町田野中,西予市,3.0,"737525-1,2,3","375,376,377",33.401812,132.536929,737525.0,1.0,45.007140
601,四国中央市三島中央1丁目,四国中央市,3,"737368-1,2,3","211,210,212",33.98279,133.540070,737368.0,1.0,737368-1,四国中央市三島中央一丁目,四国中央市,3.0,"737368-1,2,3","211,210,212",33.982627,133.539653,737368.0,1.0,42.536547
780,北宇和郡鬼北町延川,鬼北町,3,"737401-1,2,3","126,128,340(4/3)",33.299904,132.745721,737401.0,1.0,737401-1,北宇和郡鬼北町大字延川,鬼北町,3.0,"737401-1,2,3","126,128,340",33.299884,132.745283,737401.0,1.0,40.837507
1541,新居浜市庄内町1丁目,新居浜市,3,"737542-6,7,8","45,354,88(4/16)",33.956321,133.286426,737542.0,6.0,737542-6,新居浜市庄内町一丁目,新居浜市,3.0,"737542-6,7,8","45,354,88",33.956334,133.286822,737542.0,6.0,36.631479
399,新居浜市黒島2丁目,新居浜市,3,"737338-1,2,3","241,--,--",33.985738,133.343386,737338.0,1.0,737338-1,新居浜市黒島一丁目,新居浜市,3.0,"737338-1,2,3","241,359,240",33.985709,133.343032,737338.0,1.0,32.893979
771,四国中央市土居町津根②,四国中央市,3,"737400-4,5,6","55,362,318(3/7)",33.973695,133.456105,737400.0,4.0,737400-4,四国中央市土居町津根②,四国中央市,3.0,"737400-4,5,6","55,362,318",33.973869,133.455825,737400.0,4.0,32.288560
1089,喜多郡内子町本川,内子町,3,"737452-9,10,11","322,323,321(11/3)",33.5735015,132.826255,737452.0,9.0,737452-9,喜多郡内子町本川,内子町,3.0,"737452-9,10,11","322,323,321",33.573682,132.826453,737452.0,9.0,27.217146
1389,西条市黒瀬乙,西条市,3,"737518-10,11,12","193,11,9(4/23)",33.886349,133.172508,737518.0,10.0,737518-10,西条市黒瀬乙,西条市,3.0,"737518-10,11,12","193,11,9",33.886503,133.172296,737518.0,10.0,26.007701


In [46]:
df9.to_csv("distance.csv", encoding="utf_8_sig")